# Evasion Attacks and Defenses on Hugging Face Models using ART - Part 2 Defense

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import torch
import transformers

from art.estimators.classification.hugging_face import HuggingFaceClassifierPyTorch
from art.attacks.evasion import ProjectedGradientDescentPyTorch
from art.defences.trainer import AdversarialTrainerMadryPGD
from art.utils import load_dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # <-- use the GPU if available, otherwise the CPU (slow!)
# In case you work on a Mac with Apple Silicon or AMD GPU you can use 'mps' device.
device = torch.device('cuda' if torch.cuda.is_available() else 'mps')


## Dataset 


In [ ]:
# We will use a small subset of the CIFAR-10 dataset.
# For test and debugging runs you can even reduce this parameters to avoid long calculation times.
SAMPLES_PER_CLASS = 100



(x_train, y_train), (_, _), _, _ = load_dataset('cifar10')
x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
y_train = np.argmax(y_train, axis=1)

classes = np.unique(y_train)


x_subset = []
y_subset = []

for c in classes:
    indices = y_train == c
    x_subset.append(x_train[indices][:SAMPLES_PER_CLASS])
    y_subset.append(y_train[indices][:SAMPLES_PER_CLASS])

x_subset = np.concatenate(x_subset)
y_subset = np.concatenate(y_subset)

print(f'x_subset:', x_subset.dtype, x_subset.shape)
print(f'y_subset:', y_subset.dtype, y_subset.shape)

In [ ]:
label_names = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck',
]

In [ ]:
# Create state_dicts directory for saving models

if not os.path.isdir('./state_dicts'):
    os.mkdir('./state_dicts')

Set model 

# PyTorch Image Models (timm)

PyTorch Image Models (timm) is a poular repository for SOTA implementations of image models and Hugging Face is hosting many of the models and weights.

We can use timm models here with the same wrapper.

To run this part of the notebook we need to install the timm library

`pip install timm`

This notebook was ran with timm==0.9.8

In [ ]:
import timm

In [ ]:
# Start with a small ResNet Model
MODEL_NAME = 'resnet18.a1_in1k'
# Later use a bigger one: 
# MODEL_NAME = 'resnet50.a1_in1k'

In [ ]:

model = timm.create_model(MODEL_NAME, pretrained=True)
upsampler = torch.nn.Upsample(scale_factor=7, mode='nearest')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

timm_model = HuggingFaceClassifierPyTorch(
    model=model,
    loss=loss_fn,
    input_shape=(3, 32, 32),
    nb_classes=10,
    optimizer=optimizer,
    clip_values=(0, 1),
    processor=upsampler
)

In [ ]:
limit=2000
#timm_model.fit(x_subset[:limit], y_subset[:limit], nb_epochs=1)
timm_model.fit(x_subset, y_subset, nb_epochs=10)

Takes approx. 1 min / epoch for ResNet18 on MacBook Pro...


In [ ]:
output = timm_model.predict(x_subset)
preds = np.argmax(output, axis=1)

In [ ]:
timm_acc = np.mean(preds == y_subset)
print('accuracy:', timm_acc)

## Adversarial Training with ART

In [ ]:

trainer_resnet = AdversarialTrainerMadryPGD(
    classifier=timm_model,
    nb_epochs=15,
    eps=8/255,
    eps_step=1/255,
    max_iter=10
)

In [ ]:
# load saved model if it already exists, otherwise train it

model_checkpoint_path = './state_dicts/timm_%s_cifar_robust_model.pt' % MODEL_NAME
if os.path.isfile(model_checkpoint_path):
    trainer_resnet.classifier.model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))
    print('loaded model checkpoint')
else:
    trainer_resnet.fit(x_subset, y_subset, nb_epochs=5)
    torch.save(trainer_resnet.classifier.model.state_dict(), model_checkpoint_path)
    print('saved model checkpoint')

about 12 min per epoch (MacBookPro M1)

In [ ]:
outputs = trainer_resnet.classifier.predict(x_subset)
clean_preds = np.argmax(outputs, axis=1)
clean_acc = np.mean(clean_preds == y_subset)
print('ResNet clean accuracy:', clean_acc)

approx. 90 s

In [ ]:
# load adversarial samples if they already exist, otherwise generate them

adv_samples_path = './state_dicts/x_adv_robust.timm_%s_npy' % MODEL_NAME
if os.path.isfile(adv_samples_path):
    x_adv = np.load(adv_samples_path)
else:
    attacker = ProjectedGradientDescentPyTorch(trainer_resnet.classifier, eps=8/255, eps_step=1/255)
    x_adv = attacker.generate(x_subset)
    np.save(adv_samples_path, x_adv)

outputs = trainer_resnet.classifier.predict(x_adv)
adv_preds = np.argmax(outputs, axis=1)
adv_acc = np.mean(adv_preds == y_subset)
print('%s adversarial accuracy:' % MODEL_NAME, adv_acc)

### Your task
You make an observation here, which is well known in AI security: Your model suffers from being made robust... You all had basic classes in Machine Learning. Please, analyze the following aspects:
- Visualize the images that are correcly classified and those that are wrong classified
- Vary key hyperparameters (at least manually or a simple grid search), you can choose a hyperparameter tuning framework if you are familiar with it. 
- Vary the parameters of the adversarial training (parameters of class AdversarialTrainerMadryPGD, e.g. eps). Do you see an influence?
- Try with ResNet50, do you see a difference? (You will need GPUs and some time for that).

General hints:
- Consider cross-reading Madry's paper from the lecutre. 
- Work in teams

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def visualize_images(images, true_labels, predicted_labels):
    fig, axs = plt.subplots(2,num_per_row, figsize=(12, 6))
    axs = axs.flatten()

    for i, (image, true_label, predicted_label) in enumerate(zip(images, true_labels, predicted_labels)):
        image = np.transpose(image, (1, 2, 0))
        axs[i].imshow(image)
        axs[i].axis('off')
        axs[i].set_title(f'True: {true_label}\nPredicted: {predicted_label}')

    plt.tight_layout()
    plt.show()


correctly_classified_images = x_adv[adv_preds == y_subset]
correctly_classified_labels = y_subset[adv_preds == y_subset]
correctly_classified_predicted_labels = adv_preds[adv_preds == y_subset]

wrongly_classified_images = x_adv[adv_preds != y_subset]
wrongly_classified_labels = y_subset[adv_preds != y_subset]
wrongly_classified_predicted_labels = adv_preds[adv_preds != y_subset]

num_per_row = 8

# Visualize correctly classified images
visualize_images(correctly_classified_images, correctly_classified_labels, correctly_classified_predicted_labels)